In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.6.0+cu124


In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00


In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from scipy import sparse
from scipy.sparse.linalg import eigsh

In [2]:
fa_feature_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Load MCI features
fa_feature_path = "/home/snu/Downloads/Histogram_MCI_FA_20bin_updated.npy"
Histogram_feature_MCI_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Combine features and labels
X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_MCI_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_MCI_FA_array.shape[0], dtype=np.int64)
])
np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (300, 180), Labels: (300,)


In [3]:
# F = np.concatenate((Histogram_feature_CN_FA_array, Histogram_feature_MCI_FA_array), axis=0)
# F = F.astype(np.float32)
# print("Combined Shape:", F.shape)
F = X
F = F.astype(np.float32)
print("Final Shape:", F.shape)
print("Data Type:", F.dtype)

Final Shape: (300, 180)
Data Type: float32


In [4]:
def tokencut_on_features(F_array, alpha=1e-6):
    """
    Apply TokenCut clustering to feature matrix F_array (shape: N × D).
    Returns binary labels (0/1) for each node.
    """
    N, D = F_array.shape

    # 1. Normalize features row-wise
    norms = np.linalg.norm(F_array, axis=1, keepdims=True) + 1e-10
    F_norm = F_array / norms

    # 2. Construct cosine similarity matrix (fully connected)
    W = np.dot(F_norm, F_norm.T)
    W = W + alpha  # stabilizer

    # 3. Normalized Laplacian: L = I - D^{-1/2} W D^{-1/2}  where, W is the similarity matrix and D is the degree matrix
    d = np.sum(W, axis=1)
    d_inv_sqrt = np.diag(1.0 / np.sqrt(d + 1e-10))
    L = np.eye(N) - d_inv_sqrt @ W @ d_inv_sqrt

    # Sparse for efficiency
    L_sparse = sparse.csr_matrix(L)

    # 4. Compute the Fiedler vector, i.e., the second smallest eigenvector of L.
    vals, vecs = eigsh(L_sparse, k=2, which='SM')
    fiedler = vecs[:, 1]

    # 5. Threshold by mean
    threshold = fiedler.mean()
    labels = (fiedler > threshold).astype(np.int64)

    return labels, fiedler

In [5]:
labels, scores = tokencut_on_features(X)

# Evaluate
y_pred = labels
acc = accuracy_score(y, y_pred)
inv_acc = accuracy_score(y, 1 - y_pred)
if inv_acc > acc:
    y_pred = 1 - y_pred
    acc = inv_acc

prec = precision_score(y, y_pred)
rec = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# Normalize fiedler vector for log_loss
probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-10)
logloss = log_loss(y, probs)

print("===== TokenCut Results =====")
print("Accuracy Score:", acc)
print("Precision Score:", prec)
print("Recall Score:", rec)
print("F1 Score:", f1)
print("Log Loss:", logloss)

===== TokenCut Results =====
Accuracy Score: 0.7133333333333334
Precision Score: 0.7755102040816326
Recall Score: 0.6826347305389222
F1 Score: 0.7261146496815286
Log Loss: 0.6643915285975587


In [6]:
print(probs)

[0.56038965 0.45153542 0.56819757 0.25090124 0.80869724 0.27962045
 0.23182833 0.49845767 0.68776726 0.74532917 0.92072797 0.53579252
 0.5347969  0.52795231 0.68464033 0.8184013  0.18218133 0.34313183
 0.70506029 0.6376309  0.35396407 0.62699401 0.64009375 0.44472168
 0.46158665 0.64107119 0.86676501 0.36195474 0.73982243 0.55754895
 0.39699456 0.91485441 0.40767919 0.3238898  0.5324294  0.71827439
 0.57293381 0.24407741 0.07960541 0.22160597 0.4715257  0.4256618
 0.74172426 0.22423884 0.37329396 0.49033131 0.09141882 0.52734184
 0.35159649 0.3440712  0.86958134 0.61545597 0.44056599 0.3446243
 0.69397634 0.24543503 0.14501474 0.28761577 0.2211638  0.30548682
 0.5301392  0.27973884 0.44568238 0.69071665 0.31364444 0.27460204
 0.         0.48874551 0.28132637 0.42405744 0.17536995 0.38962483
 0.89556645 0.47038352 0.65435242 0.21017389 0.34427238 0.27055536
 0.30793437 0.28250027 0.48596228 0.5103926  0.18666664 0.30830102
 0.22649314 0.08148583 0.25886847 0.14565677 0.34305951 0.214313

In [7]:
num_runs = 10
acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

for run in range(num_runs):
    print(f"\n--- Run {run+1}/{num_runs} ---")
    np.random.seed(run)
    perm = np.random.permutation(X.shape[0])
    X_run = X[perm]
    y_run = y[perm]

    labels, scores = tokencut_on_features(X_run)

    # Align labels to ground truth
    y_pred = labels
    acc = accuracy_score(y_run, y_pred)
    inv_acc = accuracy_score(y_run, 1 - y_pred)
    if inv_acc > acc:
        y_pred = 1 - y_pred
        acc = inv_acc

    prec = precision_score(y_run, y_pred)
    rec = recall_score(y_run, y_pred)
    f1 = f1_score(y_run, y_pred)
    probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-10)
    logloss = log_loss(y_run, probs)

    acc_scores.append(acc)
    prec_scores.append(prec)
    rec_scores.append(rec)
    f1_scores.append(f1)
    log_losses.append(logloss)

    print(f"Run {run+1} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f} | LogLoss: {logloss:.4f}")

print("\n================ FINAL SUMMARY ================\n")
print(f"{'Metric':>15} | {'Mean':>10} ± {'Std':<10}")
print("-" * 50)
print(f"{'Accuracy':>15} | {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"{'Precision':>15} | {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"{'Recall':>15} | {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"{'F1 Score':>15} | {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"{'Log Loss':>15} | {np.mean(log_losses):.4f} ± {np.std(log_losses):.4f}")


--- Run 1/10 ---
Run 1 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 2/10 ---
Run 2 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 0.6644

--- Run 3/10 ---
Run 3 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 4/10 ---
Run 4 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 0.6644

--- Run 5/10 ---
Run 5 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 0.6644

--- Run 6/10 ---
Run 6 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 7/10 ---
Run 7 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 8/10 ---
Run 8 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 9/10 ---
Run 9 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

--- Run 10/10 ---
Run 10 | Acc: 0.7133 | Prec: 0.7755 | Rec: 0.6826 | F1: 0.7261 | LogLoss: 1.2168

================ 